In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
from thesis_v2 import dir_dict
from thesis_v2.training.training_aux import load_training_results

In [3]:
from sys import path
from os.path import join, exists, dirname
from os import makedirs



In [4]:
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200218 as param_iterator_obj,
    keygen
)

In [5]:
from numpy.linalg import norm
import numpy as np

In [6]:
from sklearn.decomposition import PCA
from pickle import dump

In [7]:


def calc_raw_stats(weight_matrix):
    # weight_matrix is N * D
    # N = number of filters
    # D = size of a (flattened) filter.
    mean_all = weight_matrix.mean(axis=1)
    std_all = weight_matrix.std(axis=1)
    print('mean_all', mean_all.mean(), mean_all.std())
    print('std_all', std_all.mean(), std_all.std())
    
def calc_normed_pca(weight_matrix, verbose=False):
    # here `normed` means PCA after normalization of weight_matrix
    
    # there are multiple ways to define `normalization`
    # 
    # a simple way is to make each flattened filter with unit norm.
    weight_matrix_norm = norm(weight_matrix, axis=1, keepdims=True)
#     assert weight_matrix_norm.shape == (256,1)
    weight_matrix_normed = weight_matrix/weight_matrix_norm
    
    # tested, works as expected.
    if verbose:
        assert np.allclose(norm(weight_matrix_normed, axis=1), 1.0)
    
    
    # then let's do PCA
    pca_obj = PCA(svd_solver='full')
    pca_obj.fit(weight_matrix_normed)
    if verbose:
        # top 10 explains 60%;
        # top 20 explains 85%;
        # looks fine to me, compared to https://doi.org/10.1101/677237
        # "Recurrent networks can recycle neural resources to flexibly trade speed for accuracy in visual recognition"
        
        print(np.cumsum(pca_obj.explained_variance_ratio_)[:20])
        print(pca_obj.components_.shape)
    
    # then let's return and save them.
    return {
        'weight_matrix_norm': weight_matrix_norm,
        'weight_matrix': weight_matrix,
        'components': pca_obj.components_,
        'explained_variance_ratio': pca_obj.explained_variance_ratio_,
    }

def construct_back_by_interleave(x1, x2, by1, by2):
    n1, m1 = x1.shape
    n2, m2 = x2.shape
    assert n1 % by1 == 0
    assert n2 % by2 == 0
    assert m1==m2 and n1//by1 == n2//by2
    
    x1 = x1.reshape(n1//by1, by1, m1)
    x2 = x2.reshape(n2//by2, by2, m2)
    return np.concatenate((x1,x2), axis=1).reshape((n1+n2, m1))


def save_one_result(filename, data):
    makedirs(dirname(filename), exist_ok=True)
    with open(filename, 'wb') as f:
        dump(data, f)

In [8]:
# create dir
global_vars = {
    'save_dir': join(dir_dict['analyses'],
                     'scripts+yuanyuan_8k_a_3day+maskcnn_polished_with_rcnn_k_bl+20200218_layer3',
                     'pca')
}

def prepare_dir():
    save_dir = global_vars['save_dir']
    if not exists(save_dir):
        makedirs(save_dir)
prepare_dir()

In [9]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [10]:
def collect_all():
    save_dir = global_vars['save_dir']
    count = 0
    verbose = True
    for idx, param in enumerate(param_iterator_obj().generate()):
        
        if param['num_layer'] < 3:
            continue
        
        if param['rcnn_bl_cls'] <= 1:
            # ignore cls=1,2 models.
            continue
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        result = load_training_results(key, return_model=False, return_checkpoint=True)
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        # load loss curve to make sure we are good in approximation.
        corr_test = ((cc_raw/cc_max_all_neurons)**2).mean()
        # only keep 0.6+ models
        if corr_test < 0.6:
            continue
        
        count += 1        
        
        def filename_gen(x):
            return join(save_dir, f'{idx}/unit_norm', f'{x}.pkl')
        
        filename_set = {'ff', 'lateral'}
        if all([exists(filename_gen(x)) for x in filename_set]):
            # right now if partially done, the partial results will be fully overwritten.
            continue
        
        # check layer one lateral.
        weight_np_ff = result['checkpoint']['model']['moduledict.bl_stack.layer_list.1.b_conv.weight'].numpy().astype(np.float64)
        weight_np_lateral = result['checkpoint']['model']['moduledict.bl_stack.layer_list.1.l_conv.weight'].numpy().astype(np.float64)

        kernel_shape = weight_np_ff.shape[2:]
        assert weight_np_ff.shape[2:] == weight_np_lateral.shape[2:] == kernel_shape
        assert weight_np_ff.ndim == weight_np_lateral.ndim == 4
        channel_low = weight_np_ff.shape[1]
        channel_high = weight_np_ff.shape[0]
        assert weight_np_lateral.shape[:2] == (channel_high, channel_high)
        
        weight_np_ff = weight_np_ff.reshape((channel_low*channel_high, kernel_shape[0]*kernel_shape[1]))
        weight_np_lateral = weight_np_lateral.reshape((channel_high*channel_high, kernel_shape[0]*kernel_shape[1]))
        
        
#         print(weight_np_lateral.shape, weight_np_ff.shape)
        
        if verbose:
            print('idx', idx)
            print('key', key)
            # show some raw stats.
            calc_raw_stats(weight_np_ff)
            calc_raw_stats(weight_np_lateral)
        pca_res_dict = dict()
        pca_res_dict['ff'] = calc_normed_pca(weight_np_ff, verbose)
        pca_res_dict['lateral'] = calc_normed_pca(weight_np_lateral, verbose)
        
        for fn in filename_set:
            save_one_result(filename_gen(fn), pca_res_dict[fn])
        
    print(count)


In [11]:
collect_all()

idx 34
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0038524930275798393 0.04557056440770497
std_all 0.04016624640380588 0.016667342225431824
mean_all 0.0033588495679547727 0.028346445699911033
std_all 0.0371010934679492 0.027218876779336567
[0.4454332  0.62314143 0.73818691 0.84152798 0.90661651 0.95339928
 0.9785207  0.99164743 1.        ]
(9, 9)
[0.29417153 0.4586617  0.60561549 0.72933621 0.82541405 0.91463234
 0.95873654 0.9903124  1.        ]
(9, 9)
idx 35
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0035767838232598267 0.04486374827909171
std_all 0.034867554532874864 0.01436335214006371
mean_all 0.00709291031420

idx 54
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0018584996110491269 0.02173001993957191
std_all 0.037250253921626716 0.014390127350221948
mean_all 0.004732063290093718 0.014207425114560072
std_all 0.03993806940980768 0.025448488288919106
[0.25954112 0.38385694 0.47938845 0.55776154 0.62882964 0.68652575
 0.73318658 0.76966375 0.80022172 0.82696406 0.84797465 0.86772665
 0.88547509 0.90044397 0.91285474 0.92338136 0.93202457 0.93956129
 0.94679081 0.95274442]
(49, 49)
[0.26497655 0.38003403 0.4547584  0.52694407 0.59139387 0.65026364
 0.69774607 0.73852213 0.77325862 0.80603334 0.83391843 0.8578755
 0.88067418 0.89745983 0.9113622  0.92249685 0.93243891 0.94010591
 0.94669375 0.9526772 ]
(49, 49)
idx 55
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k

idx 114
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l237/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0012572231179557346 0.021247767580791475
std_all 0.032605960206165595 0.010233169421860864
mean_all 0.0014671314932769232 0.012395626712100464
std_all 0.0313577567125511 0.011720047999745518
[0.23286553 0.36356183 0.45154842 0.5213011  0.58154926 0.63840259
 0.67985123 0.71545579 0.74814466 0.77575903 0.79884895 0.82172111
 0.83834383 0.85442711 0.86828536 0.87915653 0.88961898 0.89939817
 0.90901155 0.91734509]
(49, 49)
[0.17561567 0.28807845 0.39522753 0.48128746 0.55324376 0.61262253
 0.66136982 0.70347827 0.74251667 0.77343143 0.80025427 0.82668987
 0.84621862 0.86220715 0.87576471 0.88706274 0.89715808 0.9064665
 0.91363309 0.92003556]
(49, 49)
idx 115
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num

idx 163
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.002305153194606039 0.028574095522811385
std_all 0.024110998851660022 0.01040875082391459
mean_all 0.0032395764489854756 0.02736446923999001
std_all 0.02275667316122535 0.009779203519709505
[0.39098689 0.61034875 0.73724369 0.83002689 0.8887377  0.93934233
 0.9629479  0.98284885 1.        ]
(9, 9)
[0.3761779  0.548907   0.68487498 0.80916964 0.87868681 0.9371313
 0.95984694 0.98142351 1.        ]
(9, 9)
idx 164
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0025910054792025092 0.03198893167511268
std_all 0.02840978729600284 0.012154597556353676
mean_all 0.00212345387

idx 186
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actrelu/r_c2/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lmse/m_se0
mean_all -0.0006396242458119644 0.016980520646594947
std_all 0.027642250664981313 0.009940091984608353
mean_all 0.0011187544864436083 0.01594042692232225
std_all 0.0269076755804612 0.009609012819159676
[0.1980037  0.3189629  0.4221952  0.50458063 0.56417431 0.61671336
 0.66048409 0.69773774 0.72677052 0.74929989 0.76849248 0.78605028
 0.80217972 0.81399984 0.82514217 0.83526267 0.8441476  0.85181695
 0.85872248 0.86432833]
(81, 81)
[0.20237105 0.32076647 0.43428087 0.51938999 0.58093532 0.63831152
 0.68156487 0.71697244 0.74756829 0.77410928 0.79500377 0.81272847
 0.83005497 0.84016223 0.84927787 0.85788386 0.86509799 0.8713608
 0.87679703 0.88143017]
(81, 81)
idx 187
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_

...

idx 1389
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch8/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -2.4974843842073564e-05 0.04069838959042623
std_all 0.040662642277735236 0.01513528785662252
mean_all 0.0037505008589050703 0.01796419645721841
std_all 0.03195463422107734 0.02083332517992795
[0.41092652 0.54052589 0.66055197 0.73249907 0.78727454 0.82635077
 0.86192348 0.88693135 0.90838895 0.92510837 0.9400516  0.95087564
 0.96072874 0.97028737 0.97797422 0.98298024 0.98747007 0.99071193
 0.99307967 0.99499939]
(25, 25)
[0.17034305 0.32843042 0.43280825 0.52385606 0.59357588 0.65724876
 0.71443238 0.76715169 0.80745323 0.84609962 0.87180315 0.89690444
 0.91732543 0.93624558 0.95202378 0.96117458 0.96907389 0.97663385
 0.98259563 0.98735917]
(25, 25)
idx 1390
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out

idx 1453
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c3/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.000965128152397127 0.04172843808727884
std_all 0.040473690239970224 0.014358389742676052
mean_all 0.0030016803532307537 0.022942279853315827
std_all 0.03731199830078018 0.019218869120393316
[0.37132077 0.4929513  0.6002103  0.66537609 0.72752482 0.7753644
 0.81689956 0.84634188 0.87056423 0.89273839 0.91012569 0.92555596
 0.93864994 0.95029588 0.95994459 0.96729799 0.97332968 0.9786715
 0.98314656 0.98705213]
(25, 25)
[0.17222918 0.31636196 0.43381313 0.53172237 0.61450397 0.68201984
 0.7379092  0.78568801 0.82455699 0.85716569 0.8865336  0.91145295
 0.92721516 0.94082991 0.9541186  0.96299218 0.96923667 0.97484913
 0.97999927 0.98426543]
(25, 25)
idx 1454
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out

idx 1470
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch16/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_True/k_l239/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.0009421647973070612 0.025007138048268315
std_all 0.04057271571166806 0.013191790330562354
mean_all 0.0020052427848083246 0.017952045327293126
std_all 0.03838321897338942 0.018815911882210254
[0.24816051 0.36207245 0.46385662 0.53945306 0.5982856  0.6394082
 0.67712572 0.71227523 0.74449504 0.77185852 0.7969688  0.81719176
 0.83464285 0.84946359 0.86213454 0.87335935 0.8841863  0.894043
 0.90274547 0.91039407]
(81, 81)
[0.16825312 0.27992577 0.36694263 0.44124831 0.50247476 0.55896143
 0.6109294  0.65547792 0.69462162 0.73021936 0.76164319 0.78809952
 0.81179306 0.82999708 0.84554078 0.85864598 0.87080345 0.88173421
 0.89110506 0.89980218]
(81, 81)
idx 1471
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_

idx 1519
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l235/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.0013508937693014534 0.03347091770879517
std_all 0.031139746428277035 0.012195192235204422
mean_all 0.001856311922920426 0.027130406928385645
std_all 0.02807850226559018 0.01228806199874566
[0.37370153 0.53203878 0.63723088 0.72461507 0.77989127 0.82134934
 0.84940029 0.87248913 0.89167834 0.90871757 0.92284745 0.93285251
 0.94144576 0.94992054 0.95667738 0.96243669 0.96754438 0.97248715
 0.97711243 0.98154356]
(25, 25)
[0.29271759 0.44017613 0.56479036 0.65452411 0.73654628 0.79556067
 0.82619086 0.85160931 0.87516778 0.89749439 0.91262182 0.92490878
 0.93463661 0.94316407 0.9504515  0.95666966 0.96245201 0.96791074
 0.97312891 0.97807219]
(25, 25)
idx 1522
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/ou

idx 1535
key yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/s_selegacy/in_sz50/out_ch32/num_l3/k_l19/k_p3/ptavg/bn_a_fcFalse/actsoftplus/r_c4/r_psize1/r_ptypeNone/r_acccummean/ff1st_True/ff1stbba_False/k_l239/sc0.01/sm0.000005/lpoisson/m_se1
mean_all -0.0007859683475368141 0.022978862018735682
std_all 0.032647267515280666 0.012972295521012827
mean_all 0.0021482229694149875 0.022214140634128875
std_all 0.03223547543253882 0.014916819708836992
[0.25932262 0.39701233 0.50193954 0.5771447  0.63423638 0.67900972
 0.71858741 0.75407153 0.78189831 0.80489681 0.82338378 0.83891967
 0.85271523 0.86448779 0.87247551 0.87977534 0.88638347 0.89236459
 0.89763398 0.90233506]
(81, 81)
[0.22685013 0.37080494 0.47439763 0.56181752 0.6283949  0.67609923
 0.71712619 0.75250402 0.7814256  0.80754976 0.82903493 0.84611812
 0.86016836 0.87189298 0.88122698 0.88893712 0.89543359 0.90089418
 0.90570477 0.90998654]
(81, 81)
481
